In [1]:
from urllib.request import urlopen
from typing import List

from bs4 import BeautifulSoup


def get_n_grams(content: str, n: int) -> List[List[str]]:
    words = content.split(" ")
    gram_start_indices = range(len(words) - n + 1)

    n_grams = [words[i : i + n] for i in gram_start_indices]

    return n_grams

In [3]:
html_url = "http://en.wikipedia.org/wiki/Python_(programming_language)"

with urlopen(html_url) as html:
    bs = BeautifulSoup(html.read(), "html.parser")
content = bs.find("div", {"id": "mw-content-text"}).get_text()  # type: ignore

n_grams = get_n_grams(content, 2)

print(n_grams)
print(f"2-grams count is: {len(n_grams)}")

[['General-purpose', 'programming'], ['programming', 'language\n\n\nPythonParadigmMulti-paradigm:'], ['language\n\n\nPythonParadigmMulti-paradigm:', 'object-oriented,[1]'], ['object-oriented,[1]', 'procedural'], ['procedural', '(imperative),'], ['(imperative),', 'functional,'], ['functional,', 'structured,'], ['structured,', 'reflectiveDesigned\xa0byGuido'], ['reflectiveDesigned\xa0byGuido', 'van'], ['van', 'RossumDeveloperPython'], ['RossumDeveloperPython', 'Software'], ['Software', 'FoundationFirst\xa0appeared20\xa0February'], ['FoundationFirst\xa0appeared20\xa0February', '1991;'], ['1991;', '32'], ['32', 'years'], ['years', 'ago\xa0(1991-02-20)[2]Stable'], ['ago\xa0(1991-02-20)[2]Stable', 'release3.12.1\xa0\n'], ['release3.12.1\xa0\n', ''], ['', ''], ['', '/'], ['/', '7'], ['7', 'December'], ['December', '2023;'], ['2023;', '58'], ['58', 'days'], ['days', 'ago\xa0(7'], ['ago\xa0(7', 'December'], ['December', '2023)\nTyping'], ['2023)\nTyping', 'disciplineDuck,'], ['disciplineDuck,',

In [18]:
import re


def split_into_words(content: str):
    # 改行文字/引用インデックス(ex. [12])/utf-8文字をフィルタ
    content = re.sub(r"\n|[\[\d+\]]", " ", content)
    content = bytes(content, "utf-8").decode("ascii", "ignore")

    words = content.split(" ")
    return words


def get_n_grams(words: List[str], n: int):
    gram_start_indices = range(len(words) - n + 1)

    n_grams = [words[i : i + n] for i in gram_start_indices]
    return n_grams

In [19]:
html_url = "http://en.wikipedia.org/wiki/Python_(programming_language)"

with urlopen(html_url) as html:
    bs = BeautifulSoup(html.read(), "html.parser")
content = bs.find("div", {"id": "mw-content-text"}).get_text()  # type: ignore

words = split_into_words(content)
n_grams = get_n_grams(words, 2)

print(n_grams)
print(f"2-grams count is: {len(n_grams)}")

[['General-purpose', 'programming'], ['programming', 'language'], ['language', ''], ['', ''], ['', 'PythonParadigmMulti-paradigm:'], ['PythonParadigmMulti-paradigm:', 'object-oriented,'], ['object-oriented,', ''], ['', ''], ['', ''], ['', 'procedural'], ['procedural', '(imperative),'], ['(imperative),', 'functional,'], ['functional,', 'structured,'], ['structured,', 'reflectiveDesignedbyGuido'], ['reflectiveDesignedbyGuido', 'van'], ['van', 'RossumDeveloperPython'], ['RossumDeveloperPython', 'Software'], ['Software', 'FoundationFirstappeared'], ['FoundationFirstappeared', ''], ['', 'February'], ['February', ''], ['', ''], ['', ''], ['', ''], ['', ';'], [';', ''], ['', ''], ['', ''], ['', 'years'], ['years', 'ago('], ['ago(', ''], ['', ''], ['', ''], ['', '-'], ['-', ''], ['', '-'], ['-', ''], ['', ')'], [')', ''], ['', ''], ['', 'Stable'], ['Stable', 'release'], ['release', '.'], ['.', ''], ['', '.'], ['.', ''], ['', ''], ['', ''], ['', ''], ['', '/'], ['/', ''], ['', ''], ['', 'Decemb

In [20]:
from string import punctuation, whitespace


def split_into_clean_words(sentence: str):
    def is_valid_word(word: str):
        # 2文字以上の文字列とa(冠詞), I(主格名詞)を許容する
        return len(word) > 1 or word.lower() in ["a", "i"]

    def strip_word(word: str):
        return word.strip(punctuation + whitespace)

    words = sentence.split(" ")

    words = [strip_word(word) for word in words]
    words = [word for word in words if is_valid_word(word)]

    return words


def split_into_clean_sentences(content: str):
    # 改行文字/引用インデックス(ex. [12])/utf-8文字をフィルタ
    content = re.sub(r"\n|[\[\d+\]]", " ", content)
    content = bytes(content, "utf-8").decode("ascii", "ignore")

    sentences = content.split(". ")
    return sentences


def get_n_grams_from_words(words: List[str], n: int):
    gram_start_indices = range(len(words) - n + 1)

    n_grams = [words[i : i + n] for i in gram_start_indices]
    return n_grams


def get_n_grams(content: str, n: int):
    sentences = split_into_clean_sentences(content)
    n_grams: List[List[str]] = []

    for sentence in sentences:
        words = split_into_clean_words(sentence)
        n_grams_for_sentence = get_n_grams_from_words(words, n)
        n_grams += n_grams_for_sentence

    return n_grams

In [8]:
html_url = "http://en.wikipedia.org/wiki/Python_(programming_language)"

with urlopen(html_url) as html:
    bs = BeautifulSoup(html.read(), "html.parser")
content = bs.find("div", {"id": "mw-content-text"}).get_text()  # type: ignore

n_grams = get_n_grams(content, 2)
print(len(n_grams))

10278


In [16]:
from collections import Counter


def split_into_clean_upper_words(sentence: str):
    def is_valid_word(word: str):
        # 2文字以上の文字列とa(冠詞), I(主格名詞)を許容する
        return len(word) > 1 or word.lower() in ["a", "i"]

    def strip_word(word: str):
        return word.strip(punctuation + whitespace)

    # 単語の前処理
    words = sentence.split(" ")
    words = [strip_word(word) for word in words]
    words = [word for word in words if is_valid_word(word)]
    # 単語の連続性を見れればいいので、大文字小文字を識別しない
    words = [word.upper() for word in words]

    return words


def get_flatten_n_grams_from_words(words: List[str], n: int):
    gram_start_indices = range(len(words) - n + 1)

    n_grams = [words[i : i + n] for i in gram_start_indices]
    flatten_n_grams = [" ".join(n_gram) for n_gram in n_grams]

    return flatten_n_grams


def get_n_grams_counter(content: str, n: int):
    n_grams_counter: Counter[str] = Counter()

    sentences = split_into_clean_sentences(content)
    for sentence in sentences:
        words = split_into_clean_upper_words(sentence)
        n_grams_for_sentence = get_flatten_n_grams_from_words(words, n)
        n_grams_counter.update(n_grams_for_sentence)

    return n_grams_counter

In [17]:
n_grams_counter = get_n_grams_counter(content, 2)
print(n_grams_counter)

Counter({'FROM THE': 224, 'THE ORIGINAL': 215, 'ORIGINAL ON': 212, 'ARCHIVED FROM': 210, 'ON JUNE': 59, 'OF THE': 43, 'OF PYTHON': 40, 'SOFTWARE FOUNDATION': 39, 'PYTHON SOFTWARE': 39, 'IN PYTHON': 35, 'IN THE': 29, 'ON MAY': 28, 'RETRIEVED FEBRUARY': 28, 'RETRIEVED JANUARY': 27, 'THE PYTHON': 26, 'SUCH AS': 23, 'AS A': 22, 'VAN ROSSUM': 21, 'RETRIEVED MARCH': 21, 'RETRIEVED MAY': 20, 'RETRIEVED NOVEMBER': 20, 'PROGRAMMING LANGUAGE': 19, 'IS A': 19, 'ON OCTOBER': 19, 'BE USED': 18, 'RETRIEVED JULY': 18, 'ON APRIL': 18, 'RETRIEVED APRIL': 18, 'ON DECEMBER': 17, 'RETRIEVED JUNE': 17, 'CAN BE': 16, 'PYTHON IS': 15, 'IT IS': 15, 'TO THE': 15, 'TO BE': 15, 'RETRIEVED DECEMBER': 15, 'PROGRAMMING LANGUAGES': 14, 'PYTHON ENHANCEMENT': 14, 'RETRIEVED AUGUST': 14, 'WITH THE': 13, 'TO PYTHON': 13, 'FOR PYTHON': 13, 'ENHANCEMENT PROPOSALS': 13, 'RETRIEVED SEPTEMBER': 13, 'ON JANUARY': 13, 'STANDARD LIBRARY': 12, 'ON MARCH': 12, 'ROSSUM GUIDO': 12, 'ON JULY': 12, 'ON FEBRUARY': 12, 'TO A': 11, 'OF 